# Pencil Production Solution - Pyomo with GLPK
Notebook to demonstrate the use of Pyomo and the GLPK solver to solve the pencil production problem.

## Model
\begin{equation*}
\begin{array}{llllll}
\text{max} & 0.41x_e & + & 0.84x_d & & \\
\text{s.t} & & & & & \\
           & (5.56 \cdot 10^{-4})x_e & + & (5.56 \cdot 10^{-4})x_d & \le & 12\\
           & (9.72 \cdot 10^{-4})x_e & + & (9.72 \cdot 10^{-4})x_d & \le & 8.25\\
           & (3.47 \cdot 10^{-4})x_e & + & (2.43 \cdot 10^{-3})x_d & \le & 13\\
           & (2.22 \cdot 10^{-3})x_e & + & (2.78 \cdot 10^{-3})x_d & \le & 18\\
           & 0.1x_e & + & 0.1x_d & \le & 15,000\\
           & 0.094x_e & + & 0.094x_d & \le & 1,250\\
           & 0.188x_e &   &          & \le & 950\\
           &          &   & 0.219x_d & \le & 1,750\\
           & 0.175x_e &   &          & \le & 840\\
           &          &   & 0.156x_d & \le & 1,100\\
           & x_e      & + & x_d      & \le & 10,000\\
           & x_e      & + & x_d      & \le & 32,000\\
           &          &   & x_d      & \le & 20,000\\
           &          &   & x_e,x_d  & \ge & 0\\
\end{array}
\end{equation*}


## References:
- [Pencil Production Problem](https://github.com/BrockSkaggs/GrokWithBrock/blob/main/Linear%20Programming/Pencil%20Production/pencil_prod_problem.md)
- [Pyomo](http://www.pyomo.org/)
- [GLPK (GNU Linear Programming Kit)](https://www.gnu.org/software/glpk/)

## Model Solution Development

In [17]:
import pyomo.opt as po
import pyomo.environ as pe

In [18]:
model = pe.ConcreteModel(name='Pencil Production')

In [19]:
model.x_econ = pe.Var(domain = pe.NonNegativeReals)
model.x_del = pe.Var(domain = pe.NonNegativeReals)

In [20]:
obj_expr = 0.41*model.x_econ + 0.84*model.x_del
model.obj = pe.Objective(sense=pe.maximize, expr=obj_expr)

In [21]:
model.preprocessing = pe.Constraint(expr = (2/3600)*model.x_econ + (2/3600)*model.x_del <= 12)
model.curing = pe.Constraint(expr = (3.5/3600)*model.x_econ + (3.5/3600)*model.x_del <= 8.25)
model.shaping = pe.Constraint(expr = (1.25/3600)*model.x_econ + (8.75/3600)*model.x_del <= 13)
model.asbly = pe.Constraint(expr = (8/3600)*model.x_econ + (10/3600)*model.x_del <= 18)

In [22]:
model.blank = pe.Constraint(expr = 0.1*model.x_econ + 0.1*model.x_del <= 15000)
model.glue = pe.Constraint(expr = (1.5/16)*model.x_econ + (1.5/16)*model.x_del <= 1250)
model.basic_lead = pe.Constraint(expr = (3/16)*model.x_econ <= 950)
model.smooth_lead = pe.Constraint(expr = (3.5/16)*model.x_del <= 1750)
model.matte_paint = pe.Constraint(expr = (2.8/16)*model.x_econ <= 840)
model.wood_stain = pe.Constraint(expr = (2.5/16)*model.x_del <= 1100)
model.ferrule = pe.Constraint(expr = model.x_econ + model.x_del <= 10000)
model.eraser = pe.Constraint(expr = model.x_econ + model.x_del <= 32000)
model.grip = pe.Constraint(expr = model.x_del <= 20000)

In [23]:
solver = po.SolverFactory('glpk')

In [24]:
results = solver.solve(model, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\brock\AppData\Local\Temp\tmpnqpjxjkq.glpk.raw --wglp C:\Users\brock\AppData\Local\Temp\tmp2kk52a_1.glpk.glp
 --cpxlp C:\Users\brock\AppData\Local\Temp\tmpunjpt5_k.pyomo.lp
Reading problem data from 'C:\Users\brock\AppData\Local\Temp\tmpunjpt5_k.pyomo.lp'...
14 rows, 3 columns, 22 non-zeros
76 lines were read
Writing problem data to 'C:\Users\brock\AppData\Local\Temp\tmp2kk52a_1.glpk.glp'...
58 lines were written
GLPK Simplex Optimizer, v4.65
14 rows, 3 columns, 22 non-zeros
Preprocessing...
4 rows, 2 columns, 8 non-zeros
Scaling...
 A: min|aij| =  3.472e-04  max|aij| =  1.000e+00  ratio =  2.880e+03
GM: min|aij| =  6.148e-01  max|aij| =  1.627e+00  ratio =  2.646e+00
EQ: min|aij| =  3.780e-01  max|aij| =  1.000e+00  ratio =  2.646e+00
Constructing initial basis...
Size of triangular part is 4
*     0: obj =  -0.000000000e+00 inf =   0.000e+00 (2)
*     3: obj =   4.992104348e+03 inf =   0.00

In [25]:
model.display()

Model Pencil Production

  Variables:
    x_econ : Size=1, Index=None
        Key  : Lower : Value            : Upper : Fixed : Stale : Domain
        None :     0 : 1721.73913043478 :  None : False : False : NonNegativeReals
    x_del : Size=1, Index=None
        Key  : Lower : Value            : Upper : Fixed : Stale : Domain
        None :     0 : 5102.60869565217 :  None : False : False : NonNegativeReals

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 4992.104347826083

  Constraints:
    preprocessing : Size=1
        Key  : Lower : Body               : Upper
        None :  None : 3.7913043478260833 :  12.0
    curing : Size=1
        Key  : Lower : Body              : Upper
        None :  None : 6.634782608695645 :  8.25
    shaping : Size=1
        Key  : Lower : Body               : Upper
        None :  None : 12.999999999999991 :  13.0
    asbly : Size=1
        Key  : Lower : Body               : Upper
       

In [26]:
model.pprint()

2 Var Declarations
    x_del : Size=1, Index=None
        Key  : Lower : Value            : Upper : Fixed : Stale : Domain
        None :     0 : 5102.60869565217 :  None : False : False : NonNegativeReals
    x_econ : Size=1, Index=None
        Key  : Lower : Value            : Upper : Fixed : Stale : Domain
        None :     0 : 1721.73913043478 :  None : False : False : NonNegativeReals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 0.41*x_econ + 0.84*x_del

13 Constraint Declarations
    asbly : Size=1, Index=None, Active=True
        Key  : Lower : Body                                                      : Upper : Active
        None :  -Inf : 0.0022222222222222222*x_econ + 0.002777777777777778*x_del :  18.0 :   True
    basic_lead : Size=1, Index=None, Active=True
        Key  : Lower : Body          : Upper : Active
        None :  -Inf : 0.1875*x_econ : 950.0 :   True
    bla

In [28]:
qty_economy_pencils = pe.value(model.x_econ)
print(f"Qty. Economy Pencils: {qty_economy_pencils:,.2f}")

Qty. Economy Pencils: 1,721.74
